# Curate perturbation dataset with `PerturbationCurator`

Here we use pertpy's `PerturbationCurator` to ensure that a perturbatin dataset conforms to both, `CELLxGENE` (schema 5.1.0) and pertpy's defined criteria.
More specifically, the `PerturbationCurator` builds upon [cellxgene-lamin](https://github.com/laminlabs/cellxgene-lamin) and extends it by further requiring `cell_line` and `X_perturbations` columns for the perturbations.

This guide demonstrates how to curate a complex, real world perturbation dataset [McFarland et al. 2020](https://www.nature.com/articles/s41467-020-17440-w) using `PerturbationCurator`. Please have a look at [lamindb's perturbation guide](https://docs.lamin.ai/perturbation) for more details.

In [1]:
#! pip install pertpy-datasets

# using a local instance here but in practice, we use `laminlabs/pertpy-datasets`
!lamin init --storage ./test-perturbation --schema bionty,wetlab,ourprojects

→ connected lamindb: sunnyosun/test-perturbation


In [2]:
import lamindb as ln
import bionty as bt
import wetlab as wl
import ourprojects as ops
import pertpy_datasets as pts
import scanpy as sc
from datetime import timedelta

ln.track("HIRTYxL3aZc70000")

→ connected lamindb: sunnyosun/test-perturbation
→ loaded Transform('HIRTYxL3'), re-started Run('ZIxUL0yY') at 2024-12-12 17:26:45 UTC
→ notebook imports: bionty==0.53.2 lamindb==0.77.2 ourprojects==0.0.1 pertpy_datasets==0.1.0 scanpy==1.10.1 wetlab==0.36.0


In [3]:
# only for a new instance, add a source to wetlab.Compound
chebi_source = bt.Source.filter(entity="wetlab.Compound", name="chebi").first()
if not chebi_source:
    wl.Compound.add_source(
        bt.Source.filter(entity="Drug", name="chebi").first()
    )

## Dataset

In [4]:
adata = ln.Artifact.using("laminlabs/lamindata").get(uid="Xk7Qaik9vBLV4PKf0001").load()
adata.obs.head()

→ completing transfer to track Artifact('Xk7Qaik9') as input
→ mapped records: Artifact(uid='Xk7Qaik9vBLV4PKf0001')
→ transferred records: 


,depmap_id,cancer,cell_det_rate,cell_line,cell_quality,channel,disease,dose_unit,dose_value,doublet_CL1,doublet_CL2,doublet_GMM_prob,doublet_dev_imp,doublet_z_margin,hash_assignment,hash_tag,num_SNPs,organism,perturbation,perturbation_type,sex,singlet_ID,singlet_dev,singlet_dev_z,singlet_margin,singlet_z_margin,time,tissue_type,tot_reads,nperts,ngenes,ncounts,percent_mito,percent_ribo,chembl-ID
AACTGGTGTCTCTCTG,ACH-000390,True,0.093159,LUDLU-1,normal,nan,lung cancer,µM,0.1,LUDLU1_LUNG,TE14_OESOPHAGUS,2.269468e-10,0.009426,0.403316,nan,nan,481,human,trametinib,drug,Male,LUDLU1_LUNG,0.655877,14.860933,0.462273,12.351139,24,cell_line,787,1,3045,12895.0,3.202792,24.955409,CHEMBL2103875
ATAGGCTCAGATTTCG,ACH-000444,True,0.145728,LU99,normal,2,lung cancer,µM,0.5,LU99_LUNG,MCAS_OVARY,8.562908e-04,0.010173,0.188284,nan,nan,1003,human,afatinib,drug,Male,LU99_LUNG,0.762847,10.648094,0.474590,8.164565,24,cell_line,1597,1,4763,23161.0,7.473771,18.051898,CHEMBL1173655
GCCAAATCAAGCCGTC,ACH-000396,True,0.117330,J82,normal,nan,urinary bladder carcinoma,µM,0.1,J82_URINARY_TRACT,IGR1_SKIN,6.490367e-08,0.009686,1.185862,nan,nan,647,human,dabrafenib,drug,Male,J82_URINARY_TRACT,0.651059,14.740111,0.404508,11.188513,24,cell_line,1159,1,3834,18062.0,2.762706,22.085040,CHEMBL2028663
CGGAGAAGTCGCGTCA,ACH-000997,True,0.005422,HCT-15,low_quality,7,colorectal cancer,µM,0.1,HCT15_LARGE_INTESTINE,NCIH322_LUNG,NaN,0.029753,0.000794,nan,nan,30,human,gemcitabine,drug,Male,HCT15_LARGE_INTESTINE,0.970247,2.852338,0.168971,0.833455,24,cell_line,76,1,178,726.0,70.247934,5.785124,CHEMBL888
TAGTTGGAGATCGATA,ACH-000723,True,0.132708,YD-10B,low_quality,nan,head and neck cancer,nan,NaN,YD10B_UPPER_AERODIGESTIVE_TRACT,647V_URINARY_TRACT,NaN,0.156492,1.556214,nan,nan,874,human,sggpx4-2,CRISPR,Male,YD10B_UPPER_AERODIGESTIVE_TRACT,0.292802,3.272682,0.016459,0.330120,"72, 96",cell_line,2105,1,4341,20693.0,0.695887,16.242208,NaN


In [5]:
# Calculate an embedding because CELLxGENE requires one
sc.tl.pca(adata)

## Curate and register perturbations

### Genetic perturbations

In [6]:
adata.obs["genetic_perturbation"] = adata.obs["perturbation"].where(
    adata.obs["perturbation_type"] == "CRISPR", None
)

list(adata.obs["genetic_perturbation"].unique())

[nan, 'sggpx4-2', 'sglacz', 'sggpx4-1', 'sgor2j2']

In [7]:
# register genetic perturbations with their target genes
perturbations = {
    "sggpx4-1": "GPX4",
    "sggpx4-2": "GPX4",
    "sgor2j2": "OR2J2", # cutting control
}

for sg_name, gene_symbol in perturbations.items():
    perturbation = wl.GeneticPerturbation(system="CRISPR-Cas9", name=sg_name, description = "cutting control" if sg_name == "sgor2j2" else None).save()
    target = wl.PerturbationTarget(name=gene_symbol).save()
    perturbation.targets.add(target)
    gene = bt.Gene.from_source(symbol=gene_symbol, organism="human").save()
    target.genes.set([gene] if isinstance(gene, bt.Gene) else gene)

# register the negative control without targets: Non-cutting control, plasmid backbone
wl.GeneticPerturbation(system="CRISPR-Cas9", name="sglacz").save();

! record with similar name exists! did you mean to load it?


,uid,name,system,description,sequence,on_target_score,off_target_score,run_id,created_at,created_by_id
id,,,,,,,,,,
1,ddg2RoCmNNAi,sggpx4-1,CRISPR-Cas9,None,None,None,None,1,2024-12-12 17:26:56.849537+00:00,1


→ returning existing PerturbationTarget record with same name: 'GPX4'
! ambiguous validation in Bionty for 1 record: 'OR2J2'


### Compounds

In [8]:
# register compounds from CHEBI
adata.obs["compound"] = adata.obs["perturbation"].where(
    adata.obs["perturbation_type"] == "drug", None
)

wl.Compound.from_values(adata.obs["compound"]).save();

! did not create Compound records for 6 non-validated names: 'azd5591', 'brd3379', 'control', 'idasanutlin', 'prexasertib', 'taselisib'


In [9]:
# the remaining compounds are not in CHEBI and we create records for them
compounds = [wl.Compound(name=cpd) for cpd in {"azd5591", "brd3379", "control", "idasanutlin", "prexasertib","taselisib"}]
ln.save(compounds)

In [10]:
# register compound perturbations: compound + dose + duration
compound_meta = adata.obs[adata.obs["compound"].notna()][
    ["perturbation", "dose_unit", "dose_value", "time"]
].drop_duplicates()

compound_perturbation_names = {}
for _, row in compound_meta.iterrows():
    compound = wl.Compound.get(name=row["perturbation"])
    duration = row.time.split(', ')[-1]  # take the longest duration
    compound_perturbation_name = "control" if row["perturbation"] == "control" else f"{row['perturbation']}_{row.dose_value}{row.dose_unit}_{duration}h"
    compound_perturbation = wl.CompoundPerturbation(
            name=compound_perturbation_name,
            concentration=row.dose_value,
            concentration_unit=row.dose_unit,
            duration=timedelta(hours=int(duration)),
            compound=compound,
            ).save()
    compound_perturbation_names[row["perturbation"]] = compound_perturbation_name

adata.obs["compound_perturbation"] = adata.obs["compound"].map(compound_perturbation_names)

! record with similar name exists! did you mean to load it?


,uid,name,description,concentration,concentration_unit,duration,compound_id,run_id,created_at,created_by_id
id,,,,,,,,,,
1,uZddC5g2dKOb,trametinib_0.1µM_24h,None,0.1,µM,1 days,1,1,2024-12-12 17:27:09.902673+00:00,1


→ returning existing CompoundPerturbation record with same name: 'control'
! record with similar name exists! did you mean to load it?


,uid,name,description,concentration,concentration_unit,duration,compound_id,run_id,created_at,created_by_id
id,,,,,,,,,,
7,P7hRCQCDY7PY,bortezomib_2.5µM_6h,None,2.5,µM,0 days 06:00:00,6,1,2024-12-12 17:27:10.003650+00:00,1


! record with similar name exists! did you mean to load it?


,uid,name,description,concentration,concentration_unit,duration,compound_id,run_id,created_at,created_by_id
id,,,,,,,,,,
8,SeHMdmnKhIX5,brd3379_0.0µM_6h,None,0.0,µM,0 days 06:00:00,153,1,2024-12-12 17:27:10.029267+00:00,1


! record with similar name exists! did you mean to load it?


,uid,name,description,concentration,concentration_unit,duration,compound_id,run_id,created_at,created_by_id
id,,,,,,,,,,
16,frhpjw4KIBuX,idasanutlin_2.5µM_6h,None,2.5,µM,0 days 06:00:00,157,1,2024-12-12 17:27:10.118204+00:00,1


! records with similar names exist! did you mean to load one of them?


,uid,name,description,concentration,concentration_unit,duration,compound_id,run_id,created_at,created_by_id
id,,,,,,,,,,
1,uZddC5g2dKOb,trametinib_0.1µM_24h,None,0.1,µM,1 days,1,1,2024-12-12 17:27:09.902673+00:00,1
6,dIYVGDHIAl4G,trametinib_0.1µM_48h,None,0.1,µM,2 days,1,1,2024-12-12 17:27:09.989896+00:00,1


In [11]:
# optional: add protein targets to compounds
compounds_to_targets = {
    "trametinib": ("MAPK/ERK pathway", ["P36507"]),
    "afatinib": ("EGFR, HER2, HER4 signaling", ["P00533", "Q9UK79", "Q15303"]),
    "dabrafenib": ("MAPK/ERK pathway", ["P15056"]),
    "gemcitabine": ("DNA synthesis inhibition", ["P23921"]),  # No single protein target
    "navitoclax": ("Apoptosis regulation", ["P10415", "Q07812"]),
    "bortezomib": ("Proteasome pathway", ["P49721"]),
    "brd3379": ("Transcription regulation (BET proteins)", ["O60885"]),
    "JQ1": ("Transcription regulation (BET proteins)", ["O60885"]),
    "azd5591": ("Apoptosis regulation", ["Q07820"]),
    "prexasertib": ("DNA damage response", ["O14757"]),
    "taselisib": ("PI3K/AKT/mTOR pathway", ["P42336", "O00329", "P48736"]),
    "idasanutlin": ("p53 regulation", ["Q00987"]),
    "everolimus": ("mTOR pathway", ["P42345"])
}

for compound_name, compound_perturbation_name in compound_perturbation_names.items():
    if compound_perturbation_name == "control":  # No target for control
        continue
    compound_perturbation = wl.CompoundPerturbation.get(name=compound_perturbation_name)
    target = wl.PerturbationTarget(name=compounds_to_targets[compound_name][0]).save()
    proteins = bt.Protein.from_values(compounds_to_targets[compound_name][1], field=bt.Protein.uniprotkb_id).save()
    target.proteins.set(proteins)
    compound_perturbation.targets.add(target)

→ returning existing PerturbationTarget record with same name: 'MAPK/ERK pathway'
→ returning existing PerturbationTarget record with same name: 'Transcription regulation (BET proteins)'
→ returning existing PerturbationTarget record with same name: 'Apoptosis regulation'
! record with similar name exists! did you mean to load it?


,uid,name,description,run_id,created_at,created_by_id
id,,,,,,
10,MTzGXCz6,PI3K/AKT/mTOR pathway,None,1,2024-12-12 17:27:25.458552+00:00,1


## Curator

In [12]:
curator = pts.PerturbationCurator(adata, using_key="test-perturbation")
curator.validate()

→ added default value 'unknown' to the adata.obs['assay']
→ added default value 'unknown' to the adata.obs['cell_type']
→ added default value 'unknown' to the adata.obs['development_stage']
→ added default value 'unknown' to the adata.obs['donor_id']
→ added default value 'unknown' to the adata.obs['self_reported_ethnicity']
→ added default value 'cell' to the adata.obs['suspension_type']
→ added default value '' to the adata.obs['environmental_perturbation']
→ added default value '' to the adata.obs['combination_perturbation']
✓ added 16 records with Feature.name for "columns": 'assay', 'cell_type', 'development_stage', 'disease', 'donor_id', 'self_reported_ethnicity', 'sex', 'suspension_type', 'tissue_type', 'organism', 'cell_line', 'genetic_perturbation', 'compound', 'compound_perturbation', 'environmental_perturbation', 'combination_perturbation'
→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
✓ added 1869 records from 

False

In [13]:
# manually fix sex and set assay
adata.obs["sex"] = adata.obs["sex"].cat.rename_categories({"Unknown": "unknown"})
adata.obs["assay"] = "10x 3' v3"

# subset the adata to only include the validated genes
adata = adata[:, ~adata.var_names.isin(curator.non_validated["var_index"])].copy()

In [14]:
# Recreate Curator object because we are using a new adata
curator = pts.PerturbationCurator(adata, using_key="test-perturbation")
curator.validate()

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'assay'
✓ added 1 record from public with ExperimentalFactor.name for "assay": '10x 3' v3'
✓ "var_index" is validated against Gene.ensembl_gene_id
✓ "assay" is validated against ExperimentalFactor.name
• mapping "cell_type" on CellType.name
!   1 term is not validated: 'unknown'
    → fix typos, remove non-existent values, or save terms via .add_new_from("cell_type")
• mapping "development_stage" on DevelopmentalStage.name
!   1 term is not validated: 'unknown'
    → fix typos, remove non-existent values, or save terms via .add_new_from("development_stage")
• mapping "disease" on Disease.name
!   1 term is not validated: 'pancreatic cancer'
    1 synonym found: "pancreatic cancer" → "malignant pancreatic neoplasm"
    → curate synonyms via .standardize("disease")
• mapping "donor_id" on ULabel.name
!   1 term is not validated: 'unknown'
    → fix typos, remove non-existent values, or save

False

In [15]:
curator.standardize("all")

✓ standardized 1 synonym in "disease": "pancreatic cancer" → "malignant pancreatic neoplasm"
✓ standardized 2 synonyms in "sex": "Male" → "male", "Female" → "female"


In [16]:
curator.add_new_from("all")

✓ added 1 record with CellType.name for "cell_type": 'unknown'
✓ added 1 record with DevelopmentalStage.name for "development_stage": 'unknown'
✓ added 1 record with ULabel.name for "donor_id": 'unknown'
✓ added 1 record with Ethnicity.name for "self_reported_ethnicity": 'unknown'
✓ added 1 record with Phenotype.name for "sex": 'unknown'
✓ added 1 record with ULabel.name for "suspension_type": 'cell'
✓ added 1 record with ULabel.name for "tissue_type": 'cell_line'


In [17]:
curator.validate()

→ validating metadata using registries of instance test-perturbation
✓ "var_index" is validated against Gene.ensembl_gene_id
✓ "assay" is validated against ExperimentalFactor.name
✓ "cell_type" is validated against CellType.name
✓ "development_stage" is validated against DevelopmentalStage.name
✓ "disease" is validated against Disease.name
✓ "donor_id" is validated against ULabel.name
✓ "self_reported_ethnicity" is validated against Ethnicity.name
✓ "sex" is validated against Phenotype.name
✓ "suspension_type" is validated against ULabel.name
✓ "tissue_type" is validated against ULabel.name
✓ "organism" is validated against Organism.name
✓ "cell_line" is validated against CellLine.name
✓ "genetic_perturbation" is validated against GeneticPerturbation.name
✓ "compound" is validated against Compound.name
✓ "compound_perturbation" is validated against CompoundPerturbation.name
✓ "environmental_perturbation" is validated against EnvironmentalPerturbation.name
✓ "combination_perturbation" i

True

## References

In [18]:
reference = ops.Reference(
    name="Multiplexed single-cell transcriptional response profiling to define cancer vulnerabilities and therapeutic mechanism of action",
    abbr="McFarland 2020",
    url="https://www.nature.com/articles/s41467-020-17440-w",
    doi="10.1038/s41467-020-17440-w",
    text=(
        "Assays to study cancer cell responses to pharmacologic or genetic perturbations are typically "
        "restricted to using simple phenotypic readouts such as proliferation rate. Information-rich assays, "
        "such as gene-expression profiling, have generally not permitted efficient profiling of a given "
        "perturbation across multiple cellular contexts. Here, we develop MIX-Seq, a method for multiplexed "
        "transcriptional profiling of post-perturbation responses across a mixture of samples with single-cell "
        "resolution, using SNP-based computational demultiplexing of single-cell RNA-sequencing data. We show "
        "that MIX-Seq can be used to profile responses to chemical or genetic perturbations across pools of 100 "
        "or more cancer cell lines. We combine it with Cell Hashing to further multiplex additional experimental "
        "conditions, such as post-treatment time points or drug doses. Analyzing the high-content readout of "
        "scRNA-seq reveals both shared and context-specific transcriptional response components that can identify "
        "drug mechanism of action and enable prediction of long-term cell viability from short-term transcriptional "
        "responses to treatment."
    ),
).save()

## Register curated artifact

In [19]:
artifact = curator.save_artifact(description="McFarland AnnData")

!    30 unique terms (65.20%) are not validated for name: depmap_id, cancer, cell_det_rate, cell_quality, channel, dose_unit, dose_value, doublet_CL1, doublet_CL2, doublet_GMM_prob, doublet_dev_imp, doublet_z_margin, hash_assignment, hash_tag, num_SNPs, perturbation, perturbation_type, singlet_ID, singlet_dev, singlet_dev_z, ...
!    did not create Feature records for 30 non-validated names: 'cancer', 'cell_det_rate', 'cell_quality', 'channel', 'chembl-ID', 'depmap_id', 'dose_unit', 'dose_value', 'doublet_CL1', 'doublet_CL2', 'doublet_GMM_prob', 'doublet_dev_imp', 'doublet_z_margin', 'hash_assignment', 'hash_tag', 'ncounts', 'ngenes', 'nperts', 'num_SNPs', 'percent_mito', ...


In [20]:
# link the reference to the artifact
artifact.references.add(reference)

In [21]:
artifact.describe()

Artifact .h5ad/AnnData
├── General
│   ├── .uid = 'jcBPXRit3gRZ1xvb0000'
│   ├── .size = 3500720
│   ├── .hash = '_5KilLlHszvZF60lInz57w'
│   ├── .n_observations = 1000
│   ├── .path = 
│   │   /Users/sunnysun/Documents/repos.nosync/scverse/pertpy-datasets/scripts/lamindb_datasets/test-perturbation/.
│   │   lamindb/jcBPXRit3gRZ1xvb0000.h5ad
│   ├── .created_by = sunnyosun (Sunny Sun)
│   ├── .created_at = 2024-12-12 18:30:18
│   └── .transform = 'Curate perturbation dataset with `PerturbationCurator`'
├── Dataset featuers/.feature_sets
│   ├── var • 1869                  [bionty.Gene]                                                       
│   │   MAGED2                      float                                                               
│   │   CPE                         float                                                               
│   │   DDX43                       float                                                               
│   │   SPANXA2                     float                                                               
│   │   LNCPRESS1                   float                                                               
│   │   TMEM175                     float                                                               
│   │   MMD                         float                                                               
│   │   LINC01834                   float                                                               
│   │   DYNC2I1                     float                                                               
│   │   CCRL2                       float                                                               
│   │   MRPS6                       float                                                               
│   │   KCNH1                       float                                                               
│   │   TBILA                       float                                                               
│   │   SLC5A7                      float                                                               
│   │   GSX2                        float                                                               
│   │   ATP5MG                      float                                                               
│   │   TMEM190                     float                                                               
│   │   GTF2H1                      float                                                               
│   └── obs • 16                    [Feature]                                                           
│       assay                       cat[bionty.ExperimentalF…  10x 3' v3                                
│       cell_line                   cat[bionty.CellLine]       22Rv1, 253J-BV, 42-MG-BA, 639-V, 647-V, …
│       cell_type                   cat[bionty.CellType]       unknown                                  
│       compound                    cat[wetlab.Compound]       JQ1, afatinib, azd5591, bortezomib, brd3…
│       compound_perturbation       cat[wetlab.CompoundPertu…  JQ1_2.5µM_24h, afatinib_0.5µM_24h, azd55…
│       development_stage           cat[bionty.Developmental…  unknown                                  
│       disease                     cat[bionty.Disease]        bile duct cancer, bone cancer, brain can…
│       donor_id                    cat[ULabel]                unknown                                  
│       genetic_perturbation        cat[wetlab.GeneticPertur…  sggpx4-1, sggpx4-2, sglacz, sgor2j2      
│       organism                    cat[bionty.Organism]       human                                    
│       self_reported_ethnicity     cat[bionty.Ethnicity]      unknown                                  
│       sex                         cat[bionty.Phenotype]      female, male, unknown                    
│       suspension_type             cat[ULabel]                cell                              